<a href="https://colab.research.google.com/github/SarahBenabdallah/Deep-Learning-Neural-Networks-Parallelization-/blob/main/PFA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installations and Imports**




In addition to the pre-installed packages like numpy, pandas, matplotlib, keras, tensorflow, we'll install concise, a keras extension for regulatory genomics https://www.cmm.in.tum.de/public/docs/concise/

In [ ]:
!pip install tensorflow-gpu
!pip install -U tensorboard
!pip install pysster

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pysster.One_Hot_Encoder import One_Hot_Encoder
one = One_Hot_Encoder('ACGT')
from google.colab import files






Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached keras-2.9.0-py2.py3-none-any.whl (1.6 MB)
     |████████████████████████████████| 15.7 MB 16.5 MB/s 
  Using cached tensorboard-2.9.0-py3-none-any.whl (5.8 MB)
  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Succes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Keras-2.2.5-py2.py3-none-any.whl (336 kB)
  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Using cached tensorflow_estimator-1.15.1-py2.py3-none-any.whl (503 kB)
  Using cached tensorboard-1.15.0-py3-none-any.whl (3.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.0
    Uninstalling tensorboard-2.9.0:
      Successfully uninstalled tensorboard-2.9.0
  Attempti

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# Load the TensorBoard notebook extension
%load_ext tensorboard

**Get Data**

We are going to use simulated data of 10,000 500 bp long sequences with the positive set containing an instance of the TAL1 motif: 
TAL1 known4 and the negative set will be random sequences. 
The data were simulated using simDNA https://github.com/kundajelab/simdna by Johnny Israeli
and were deposited to https://github.com/kundajelab/dragonn.


In [ ]:
#get the dataset npz version
!wget 'https://github.com/kundajelab/dragonn/raw/master/paper_supplement/simulation_data/GC_fraction0.4motif_nameTAL1_known4num_neg10000num_pos10000seq_length500.npz'

--2022-05-31 09:49:04--  https://github.com/kundajelab/dragonn/raw/master/paper_supplement/simulation_data/GC_fraction0.4motif_nameTAL1_known4num_neg10000num_pos10000seq_length500.npz
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kundajelab/dragonn/master/paper_supplement/simulation_data/GC_fraction0.4motif_nameTAL1_known4num_neg10000num_pos10000seq_length500.npz [following]
--2022-05-31 09:49:05--  https://raw.githubusercontent.com/kundajelab/dragonn/master/paper_supplement/simulation_data/GC_fraction0.4motif_nameTAL1_known4num_neg10000num_pos10000seq_length500.npz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting re

**Load the Data**

In [ ]:
def load_simulated_data(path):
    """Load the simulated dataset
    
    Args:
      path: path to the .npz file c
    """
    data = np.load(path)

    x_train = data["X_train"].squeeze(1).swapaxes(1,2)
    x_valid = data["X_valid"].squeeze(1).swapaxes(1,2)
    y_train = data['y_train']
    y_valid = data['y_valid']
    return (x_train, y_train), (x_valid, y_valid)

In [ ]:
ls

df_test.csv
df_train.csv
GC_fraction0.4motif_nameTAL1_known4num_neg10000num_pos10000seq_length500.npz
GC_fraction0.4motif_nameTAL1_known4num_neg10000num_pos10000seq_length500.npz.1
logs/
sample_data/
x_test.csv
x_train.csv
y_test.csv
y_train.csv


and load the first file. The data is splitted into training dataset, on wchih the model will be fitted, and a test dataset on which its performance will be assessed

In [ ]:
(x_train, y_train), (x_test, y_test) = load_simulated_data("GC_fraction0.4motif_nameTAL1_known4num_neg10000num_pos10000seq_length500.npz")

**Print data shapes** 

In [ ]:
print('x_train shape : ' ,x_train.shape)
print('y_train shape : ' ,y_train.shape)
print('x_test shape : ' ,x_test.shape)
print('y_test shape : ' ,y_test.shape)

x_train shape :  (12800, 500, 4)
y_train shape :  (12800, 1)
x_test shape :  (3200, 500, 4)
y_test shape :  (3200, 1)


As you can see, there are 12.8k training examples and 3.2k test examples.

**Transform one hot encoded sequences to DNA strings**


In [ ]:
x_train_encoded = [one.decode(x) for x in x_train]
x_test_encoded = [one.decode(x) for x in x_test]

**Transform numpy train data to csv files**

In [ ]:
x_train_df = pd.DataFrame(x_train_encoded)
x_train_df.to_csv('x_train.csv')
y_train_df = pd.DataFrame(y_train)
y_train_df.to_csv('y_train.csv')
df_train = pd.concat([x_train_df, y_train_df], axis= 1)
df_train.to_csv('df_train.csv')

**Transform numpy test data to csv files**

In [ ]:
x_test_df = pd.DataFrame(x_test_encoded)
x_test_df.to_csv('x_test.csv')
y_test_df = pd.DataFrame(y_test)
y_test_df.to_csv('y_test.csv')
df_test = pd.concat([x_test_df, y_test_df], axis= 1)
df_test.to_csv('df_test.csv')

**Visualize train DF**

In [ ]:
df_train.head()

**Visualize test DF**

In [ ]:
df_test.head()

**Download train data**

In [ ]:
files.download('x_train.csv')
files.download('y_train.csv')
files.download('df_train.csv')


**Download test data**

In [ ]:
files.download('x_test.csv')
files.download('y_test.csv')
files.download('df_test.csv')

**Analyze Data**

In [ ]:
y_train.mean()

In [ ]:
y_test.mean()

As the means are close to 0.5, there are balanced classes, i.e. roughly the same number of positive and negative instances. This ratio is the same for training and testing. This is an ideal situation for training a classifier.

In [ ]:
x_train[0][:10]

As you can see the input is the one-hot-encoded DNA sequence

**Define the model**

We'll use Keras (https://keras.io), a popular deep learning library wrapping frameworks like TensorFlow to define and train a neural network. 

In [ ]:
#IMPORTS:
from keras.models import Sequential
import keras.layers as kl
from keras.callbacks import EarlyStopping, History, TensorBoard


Define the model architecture.
It can be simply define as a sequence of layers using keras.models.Sequential

We will now implement a neural network consisting of 16 convolutional filters whose outputs are transfomed by the ReLU activation function ( ReLU(x)=max(0,x) ) . On each of the 16 vectors, we keep the maximal value ("max pooling") giving a us 16-long vector. We then perform a linear transformation of this 16-long vector into a single scalar ("Dense layer") which we map to the  [0,1]  interval with sigmoid activation function  sigm(x)=11+exp(−x)  . The interpratation of the output is the probability that the sequence is bound by the transcription factor TAL1.

In [ ]:
model = Sequential([kl.Conv1D(filters=16, kernel_size=15, activation='relu', input_shape=x_train.shape[1:]),
                    kl.GlobalMaxPooling1D(),
                    kl.Dense(units=1),
                    kl.Activation('sigmoid')
                   ])

Compile the model. We need to specify which optimizer do we use, what is the loss function and what additional metrics we want to use. In our case we use the adam optimizer, the binary crossentropy as a loss function and accuracy as a metric.

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

Visualize model layers

In [ ]:
model.summary()

**Train the model**

 Making sure our results are reproducible.The model is namely initialized using random weights

In [ ]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [ ]:
tf_callback = TensorBoard(log_dir="logs", histogram_freq=1)
history = model.fit(x_train, 
                    y_train, 
                    validation_split=0.2,  # use 20% of the data as the validation set
                    callbacks=[EarlyStopping(patience=3),   # If val_loss doesn't decrease after 3 epochs, stop training
                               History(),
                               tf_callback],
                    batch_size=512,  # On every optimization step, we use 512 examples from the training data
                    epochs=100)  # 1 epoch = one full pass through the data




Train on 10240 samples, validate on 2560 samples








Epoch 1/100
10240/10240 [==============================] - 3s 276us/step - loss: 0.6977 - acc: 0.4961 - val_loss: 0.6919 - val_acc: 0.5305

Epoch 2/100
10240/10240 [==============================] - 3s 258us/step - loss: 0.6923 - acc: 0.5144 - val_loss: 0.6901 - val_acc: 0.5363
Epoch 3/100
10240/10240 [==============================] - 5s 472us/step - loss: 0.6890 - acc: 0.5473 - val_loss: 0.6866 - val_acc: 0.5719
Epoch 4/100
10240/10240 [==============================] - 4s 430us/step - loss: 0.6839 - acc: 0.5685 - val_loss: 0.6805 - val_acc: 0.5973
Epoch 5/100
10240/10240 [==============================] - 2s 241us/step - loss: 0.6760 - acc: 0.6232 - val_loss: 0.6718 - val_acc: 0.6734
Epoch 6/100
10240/10240 [==============================] - 2s 242us/step - loss: 0.6651 - acc: 0.6939 - val_loss: 0.6612 - val_acc: 0.7281
Epoch 7/100
10240/10240 [==============================] - 2s 242us/step - loss: 0.6518 - acc: 0.7394 - v

We use early stopping as a form of regularization used to avoid overfitting when training a learner with an iterative method, such as gradient descent. Such methods update the learner so as to make it better fit the training data with each iteration.

**Evaluate the model**

Once the model has been trained, one should evaluate the model on the held-out test set using different metrics. In our case we use:

*   The **auc** metric from **concise.eval_metrics**, it represents the Area Under the Curve (AUC) is the measure of the ability of a classifier to distinguish between classes and is used as a summary of the ROC curve. The higher the AUC, the better the performance of the model at distinguishing between the positive and negative classes.
*   The **aupcr** metric from **concise.eval_metrics**, it rpresents the area under the precision-recall curve (AUPRC) is a useful performance metric for imbalanced data in a problem setting where you care a lot about finding the positive examples.




**Visualize the training history**


As the model sees more and more data, it becomes more accurate on the training set and also on the validation set.

In [ ]:
%tensorboard --logdir logs